<a href="https://colab.research.google.com/github/IvonLuiz/Sprint_Automator-Preenchimento_de_Planilhas/blob/google_sheets_branch/Sprint_Automator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sprint Automator

## Overview

The Sprint Automator is a project designed to automate the task of populating cells with information related to tasks completed during sprints.

## How to Use

Here is a step by step to be able to connect to your spreedshet. You can also follow this tutorial if you prefer: https://www.youtube.com/watch?v=P16uA1Hl4DI

1. You will need a google service account. After logging in to https://console.cloud.google.com with your email, follow the steps:
- Create a new project if you dont have one.
- Activate Google Sheets API and Google Drive API.
- Click on APIs and services, credentials, create credentials, service account.
- Fill the blanks and give owner role
- After creating the new serice account, click actions, manage keys, add key, create new key, JSON and create.

The json file will be downloaded with the creadentials to acess the service account. Put the file on your google drive desired folder and rename it to credentials.json

You also will need to give permission to acesse the google spredsheet manually. Go to your spredsheet and click on share. Copy the email id from your service account. This can be found on: APIs and services, Google Sheets API, credentials, under service account email will be something like: "xxxxxx@yyyyyy.iam.serviceaccount.com". Copy it and paste it to send the request to share the spredsheet as an editor.

# Librarys

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
!pip install gspread==5.12.3

!pip install oauth2client --upgrade

In [31]:
import pandas as pd
import gspread
import re
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Variables

Also, you will need to fill in the following variables with the correct data for the code to run. Place the directories correctly to avoid errors.

In [32]:
# Credentials from the google service account with the Spreedsheet API active
credentials_path = '/content/drive/MyDrive/Colab Notebooks/credentials/credentials.json'

# Path to the task file from bitrix on your google drive
path_to_file = '/content/drive/MyDrive/Colab Notebooks/sheets/tasks_tlog.xls'

# Chose the spreedsheet title to update
# Opitions:
# [Edenred Ticket log] Controle de projeto ATIVO - v2
# [Edenred CSC] Controle de projeto ATIVO - v2
# [Edenred GTA] Controle de projeto ATIVO
# [Edenred GTA - Sustentação PMO] Controle de projeto ATIVO - v2
# [Edenred GTA - Motor de Crédito] Controle de projeto ATIVO

spreedsheet_title = '[Edenred Ticket log] Controle de projeto ATIVO - v2'

In [33]:
# If error SpreadsheetNotFound: <Response [200]>, you probably didn't share the spredsheet to the google service account email
gc = gspread.service_account(credentials_path)
database = gc.open(spreedsheet_title)   # Worksheet we will be working on

In [34]:
# Tasks worksheet ('Tarefas') is the main worksheet we need to update
tasks_wks = database.worksheet('Tarefas')
all_tasks_values = tasks_wks.get_values()

# Get the last row from sheets and extract it's sprint number to get the new one
sprint_number = int(all_tasks_values[len(all_tasks_values)-1][0]) + 1

# If you want to put the sprint number manually, uncomment this next line
# sprint_number = 51  # Chose the sprint number to update

In [35]:
sprint_number

51

# Auxiliar Functions

In [8]:
def clear_and_append_worksheet(worksheet, clear_matrix=['A2:ZZ300'], data_to_append=0):
        worksheet.batch_clear(clear_matrix)
        worksheet.append_rows(data_to_append)


def find_column_index(worksheet, header_name):
        df = pd.DataFrame(worksheet.get_all_records())
        return df.columns.get_loc(header_name) + 1


def apply_formula_to_column(worksheet, formula, column, start_row, end_row):
        # Apply the formula to each cell in the specified range
        for row in range(start_row, end_row + 1):
                formula_string = formula.format(column, row)
                cell_range = f"{column}{row}"
                worksheet.update(cell_range, formula_string, raw=False)


# Updated function to apply the formula to a specific column and range
def apply_formula_to_column_name(worksheet, formula, column, start_row, end_row):
        # Find the column index based on the header name
        column_index = find_column_index(worksheet, column)
        # Apply the formula to each cell in the specified range
        for row in range(start_row, end_row + 1):
                formula_string = formula.format(column_index, row)
                cell_range = f"{chr(65 + column_index - 1)}{row}"  # Convert column index to letter
                worksheet.update(cell_range, formula_string, raw=False)


def apply_value_to_column_name(worksheet, value, column_name, start_row, end_row):
    # Find the column index based on the header name
    column_index = find_column_index(worksheet, column_name)

    # Apply the value to each cell in the specified range
    for row in range(start_row, end_row + 1):
        cell_range = f"{chr(65 + column_index - 1)}{row}"  # Convert column index to letter
        worksheet.update(cell_range, value, raw=False)


def find_related(dataframe, search_column, target_word, return_column):
        contains_word = dataframe[search_column].str.contains(target_word, case=False)

        # Get the rows where the target_word is present
        rows_with_word = dataframe[contains_word]

        if not rows_with_word.empty:
                # Get the values from the specified return_column where the target_word is present
                values_from_return_column = rows_with_word[return_column]

                return values_from_return_column
        else:
                return pd.DataFrame(), None


def get_unique_relations(worksheet, column1, column2):
    # Obter os valores das colunas especificadas
    column1_values = worksheet.col_values(find_column_index(worksheet, column1))
    column2_values = worksheet.col_values(find_column_index(worksheet, column2))

    # Criar um dicionário para armazenar as relações únicas
    relations = {}

    # Iterar sobre os valores das colunas e criar as relações no dicionário
    for val1, val2 in zip(column1_values[1:], column2_values[1:]):  # Ignorar o cabeçalho
        if val1 not in relations:
            relations[val1] = [val2]
        elif val2 not in relations[val1]:
            relations[val1].append(val2)

    print(relations)
    return relations


def update_df_with_relations(df, column_name_task, column_to_update, worksheet):
    column_values = df[column_name_task]
    relations_dict = get_unique_relations(worksheet, column_name_task, column_to_update)
    print("Relations found: ")

    for i in range(len(column_values)):
        key = column_values[i]
        if key in relations_dict:
            # Encontrar a última relação que não seja um valor nulo ou vazio
            latest_value = next((value for value in reversed(relations_dict[key]) if value not in (None, '')), None)
            # Atualizar o DataFrame com os novos valores
            df.at[i, column_to_update] = latest_value



# Sprint data


In [9]:
tables = pd.read_html(path_to_file)
tasks_file = pd.concat(tables, ignore_index=True)

# Access the 'Task' column of the DataFrame
task_column = tasks_file['Task']

# Check if 'sprint_number' is present in any task
contains_sprint = task_column.str.contains('Sprint {:d}'.format(int(sprint_number)))
rows_from_sprint = tasks_file[contains_sprint]

# Display the rows where 'Sprint chosen' is present
print(f"Primeiras linhas das tarefas da Sprint: {sprint_number}. Esta contem {len(rows_from_sprint)} tasks")
rows_from_sprint.head()


Primeiras linhas das tarefas da Sprint: 51. Esta contem 24 tasks


,ID,Task,Description,Active,Deadline,Created by,Assignee,Participants,Observers,Status,...,Company,Deal,CRM,Parent task ID,Parent task name,Produto,Entrega,CTI,Id task atrasada,Tempo Efetivo Repasse
1030,294772,[Edenred Ticket Log- Data Squad - Sprint 51] R...,NaN,13/05/2024 11:11:17,24/05/2024 19:00:00,Luiza Pellegrino,Thalis Pasolini Trabach,NaN,"Fernando de Souza, Vitor Gerber Weiss, Luiza P...",Pending,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06.04.01-99,NaN,NaN
1031,294774,[Edenred Ticket Log- Data Squad - Sprint 51] R...,NaN,13/05/2024 11:11:32,24/05/2024 19:00:00,Luiza Pellegrino,Juliana Gabriela dos Santos,NaN,"Fernando de Souza, Vitor Gerber Weiss, Luiza P...",In Progress,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06.04.01-99,NaN,NaN
1032,294778,[Edenred Ticket Log- Data Squad - Sprint 51] R...,NaN,13/05/2024 11:12:12,24/05/2024 19:00:00,Luiza Pellegrino,Thaís Félix dos Santos,NaN,"Fernando de Souza, Luiza Pellegrino, Higino Ne...",In Progress,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06.04.01-99,NaN,NaN
1033,294780,[Edenred Ticket Log- Data Squad - Sprint 51] R...,NaN,13/05/2024 11:12:30,24/05/2024 19:00:00,Luiza Pellegrino,Hevans Vinicius Pereira,NaN,"Fernando de Souza, Vitor Gerber Weiss, Luiza P...",In Progress,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06.04.01-99,NaN,NaN
1034,294782,[Edenred Ticket Log- Data Squad - Sprint 51] A...,NaN,13/05/2024 11:12:54,24/05/2024 19:00:00,Luiza Pellegrino,Viviane Pagnussat Cechetti,NaN,"Fernando de Souza, Luiza Pellegrino, Higino Ne...",In Progress,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06.05.04-00,NaN,NaN


# Input worksheet

Connect to google sheets with credentials

In [8]:
# Input_Bitrix worksheet
input_wks = database.worksheet('Input_Conversão')
input_wks

# Clear cells except header and add new data
clear_and_append_worksheet(input_wks, ['A2:ZZ300'], rows_from_sprint.fillna('').values.tolist())

# Output worksheet

## Main formulas

Output_Bitrix worksheet

In [9]:
output_wks = database.worksheet('Output_Conversão')
output_wks

<Worksheet 'Output_Conversão' id:1075506706>

To use the formulas below correctly, we need to find some relative places on the strings from input tables, because this will vary based on the diferrent spreedsheet.

In [10]:
sprint_formulas = []
task_formulas = []
row_number = 1
sprint_size = len(str(sprint_number))

for task_string in rows_from_sprint['Task']:
    match = re.search(str(sprint_number), str(task_string))
    row_number += 1
    if match:
      sprint_formulas.append(f"=EXT.TEXTO(Input_Conversão!B{row_number}; {match.start() + sprint_size - 1}; 2)" )
      task_formulas.append(f"=EXT.TEXTO(Input_Conversão!B{row_number}; {match.start() + 3 + sprint_size}; 5000)")
    else:
        print(f'No match of sprint number was found on {task_string}. Verify your table or your variable choices. Appending {sprint_number} raw instead.')
        sprint_formulas.append(sprint_number)

print(sprint_formulas)

['=EXT.TEXTO(Input_Conversão!B2; 42; 2)', '=EXT.TEXTO(Input_Conversão!B3; 42; 2)', '=EXT.TEXTO(Input_Conversão!B4; 42; 2)', '=EXT.TEXTO(Input_Conversão!B5; 42; 2)', '=EXT.TEXTO(Input_Conversão!B6; 42; 2)', '=EXT.TEXTO(Input_Conversão!B7; 42; 2)', '=EXT.TEXTO(Input_Conversão!B8; 42; 2)', '=EXT.TEXTO(Input_Conversão!B9; 42; 2)', '=EXT.TEXTO(Input_Conversão!B10; 42; 2)', '=EXT.TEXTO(Input_Conversão!B11; 42; 2)', '=EXT.TEXTO(Input_Conversão!B12; 42; 2)', '=EXT.TEXTO(Input_Conversão!B13; 42; 2)', '=EXT.TEXTO(Input_Conversão!B14; 42; 2)', '=EXT.TEXTO(Input_Conversão!B15; 42; 2)', '=EXT.TEXTO(Input_Conversão!B16; 42; 2)', '=EXT.TEXTO(Input_Conversão!B17; 42; 2)', '=EXT.TEXTO(Input_Conversão!B18; 42; 2)', '=EXT.TEXTO(Input_Conversão!B19; 42; 2)', '=EXT.TEXTO(Input_Conversão!B20; 42; 2)', '=EXT.TEXTO(Input_Conversão!B21; 42; 2)', '=EXT.TEXTO(Input_Conversão!B22; 42; 2)', '=EXT.TEXTO(Input_Conversão!B23; 42; 2)', '=EXT.TEXTO(Input_Conversão!B24; 42; 2)', '=EXT.TEXTO(Input_Conversão!B25; 42; 2)']

Update the worksheet with the formula in column A from row 2 to the end of rows_from_sprint

In [11]:
spreedsheet_title

'[Edenred Ticket log] Controle de projeto ATIVO - v2'

In [12]:
data_output = {
  'Sprint': sprint_formulas,
  'ID': [f"=Input_Conversão!A{row_number}" for row_number in range(2, len(rows_from_sprint) + 2)],
  'CTI': [f"=Input_Conversão!AE{row_number}" for row_number in range(2, len(rows_from_sprint) + 2)],
  'Classificação': None,
  'Task': task_formulas,
  'Descricao': [f"=Input_Conversão!C{row_number}" for row_number in range(2, len(rows_from_sprint) + 2)],
  'Entrega': [f"=Input_Conversão!AD{row_number}" for row_number in range(2, len(rows_from_sprint) + 2)],
  'Produto': [f"=Input_Conversão!AC{row_number}" for row_number in range(2, len(rows_from_sprint) + 2)],
  'Responsavel': [f"=Input_Conversão!G{row_number}" for row_number in range(2, len(rows_from_sprint) + 2)],
  'Marcador': [f"=Input_Conversão!U{row_number}" for row_number in range(2, len(rows_from_sprint) + 2)],
  'Função': None,
  'Bitrix?': ["Sim" for _ in range(2, len(rows_from_sprint) + 2)],
  'Status': [f"=Input_Conversão!J{row_number}" for row_number in range(2, len(rows_from_sprint) + 2)],
  'Inicio': None,
  'Conclusao': None,
  'Tempo estimado (horas)': [f"=ESQUERDA(Input_Conversão!P{row_number}; 2)" for row_number in range(2, len(rows_from_sprint) + 2)],
  'Tempo Efetivo (horas)': [f"= Input_Conversão!T{row_number} * 24" for row_number in range(2, len(rows_from_sprint) + 2)],
  'Tempo Efetivo Repasse (horas)': [f"=Input_Conversão!AF{row_number}" for row_number in range(2, len(rows_from_sprint) + 2)],
  'Id task atrasada': [f"=Input_Conversão!AG{row_number}" for row_number in range(2, len(rows_from_sprint) + 2)],
}


df_output = pd.DataFrame(data_output)

df_output.head()

,Sprint,ID,CTI,Classificação,Task,Descricao,Entrega,Produto,Responsavel,Marcador,Função,Bitrix?,Status,Inicio,Conclusao,Tempo estimado (horas),Tempo Efetivo (horas),Tempo Efetivo Repasse (horas),Id task atrasada
0,=EXT.TEXTO(Input_Conversão!B2; 42; 2),=Input_Conversão!A2,=Input_Conversão!AE2,None,=EXT.TEXTO(Input_Conversão!B2; 46; 5000),=Input_Conversão!C2,=Input_Conversão!AD2,=Input_Conversão!AC2,=Input_Conversão!G2,=Input_Conversão!U2,None,Sim,=Input_Conversão!J2,None,None,=ESQUERDA(Input_Conversão!P2; 2),= Input_Conversão!T2 * 24,=Input_Conversão!AF2,=Input_Conversão!AG2
1,=EXT.TEXTO(Input_Conversão!B3; 42; 2),=Input_Conversão!A3,=Input_Conversão!AE3,None,=EXT.TEXTO(Input_Conversão!B3; 46; 5000),=Input_Conversão!C3,=Input_Conversão!AD3,=Input_Conversão!AC3,=Input_Conversão!G3,=Input_Conversão!U3,None,Sim,=Input_Conversão!J3,None,None,=ESQUERDA(Input_Conversão!P3; 2),= Input_Conversão!T3 * 24,=Input_Conversão!AF3,=Input_Conversão!AG3
2,=EXT.TEXTO(Input_Conversão!B4; 42; 2),=Input_Conversão!A4,=Input_Conversão!AE4,None,=EXT.TEXTO(Input_Conversão!B4; 46; 5000),=Input_Conversão!C4,=Input_Conversão!AD4,=Input_Conversão!AC4,=Input_Conversão!G4,=Input_Conversão!U4,None,Sim,=Input_Conversão!J4,None,None,=ESQUERDA(Input_Conversão!P4; 2),= Input_Conversão!T4 * 24,=Input_Conversão!AF4,=Input_Conversão!AG4
3,=EXT.TEXTO(Input_Conversão!B5; 42; 2),=Input_Conversão!A5,=Input_Conversão!AE5,None,=EXT.TEXTO(Input_Conversão!B5; 46; 5000),=Input_Conversão!C5,=Input_Conversão!AD5,=Input_Conversão!AC5,=Input_Conversão!G5,=Input_Conversão!U5,None,Sim,=Input_Conversão!J5,None,None,=ESQUERDA(Input_Conversão!P5; 2),= Input_Conversão!T5 * 24,=Input_Conversão!AF5,=Input_Conversão!AG5
4,=EXT.TEXTO(Input_Conversão!B6; 42; 2),=Input_Conversão!A6,=Input_Conversão!AE6,None,=EXT.TEXTO(Input_Conversão!B6; 46; 5000),=Input_Conversão!C6,=Input_Conversão!AD6,=Input_Conversão!AC6,=Input_Conversão!G6,=Input_Conversão!U6,None,Sim,=Input_Conversão!J6,None,None,=ESQUERDA(Input_Conversão!P6; 2),= Input_Conversão!T6 * 24,=Input_Conversão!AF6,=Input_Conversão!AG6


## Dates

Get the smallest date as start of sprint

In [13]:
# Converta a coluna 'Created on' para o formato de data
dates = pd.to_datetime(rows_from_sprint['Created on'], format='%d/%m/%Y %H:%M:%S')

# Encontre a menor data na coluna 'Created on'
start_date = dates.min()

start_date = start_date.strftime('%d/%m/%Y')

print(f"A menor data da sprint em 'Created on' é: {start_date}")

A menor data da sprint em 'Created on' é: 13/05/2024


In [14]:
# Crie uma lista com a menor data repetida para o tamanho de len(rows_from_sprint)
start_dates_list = [start_date] * len(rows_from_sprint)

start_dates_str_list = [str(date) for date in start_dates_list]

df_output['Inicio'] = start_dates_str_list

Conclusion Date

In [15]:
# Adicione 10 dias à data de início
conclusion_date = pd.to_datetime(start_date, format='%d/%m/%Y') + pd.Timedelta(days=11)

# Formate a data final como string no formato desejado
conclusion_date_str = conclusion_date.strftime('%d/%m/%Y')

# Crie uma lista com a data de conclusão repetida para o tamanho de len(rows_from_sprint)
conclusion_dates_list = [conclusion_date_str] * len(rows_from_sprint)

# Crie uma lista com a data de conclusão repetida para o tamanho de len(rows_from_sprint)
conclusion_dates_str_list = [str(date) for date in conclusion_dates_list]

df_output['Conclusao'] = conclusion_dates_str_list
print(f'Data de conclusão da sprint: {conclusion_date}')
df_output.head()

Data de conclusão da sprint: 2024-05-24 00:00:00


,Sprint,ID,CTI,Classificação,Task,Descricao,Entrega,Produto,Responsavel,Marcador,Função,Bitrix?,Status,Inicio,Conclusao,Tempo estimado (horas),Tempo Efetivo (horas),Tempo Efetivo Repasse (horas),Id task atrasada
0,=EXT.TEXTO(Input_Conversão!B2; 42; 2),=Input_Conversão!A2,=Input_Conversão!AE2,None,=EXT.TEXTO(Input_Conversão!B2; 46; 5000),=Input_Conversão!C2,=Input_Conversão!AD2,=Input_Conversão!AC2,=Input_Conversão!G2,=Input_Conversão!U2,None,Sim,=Input_Conversão!J2,13/05/2024,24/05/2024,=ESQUERDA(Input_Conversão!P2; 2),= Input_Conversão!T2 * 24,=Input_Conversão!AF2,=Input_Conversão!AG2
1,=EXT.TEXTO(Input_Conversão!B3; 42; 2),=Input_Conversão!A3,=Input_Conversão!AE3,None,=EXT.TEXTO(Input_Conversão!B3; 46; 5000),=Input_Conversão!C3,=Input_Conversão!AD3,=Input_Conversão!AC3,=Input_Conversão!G3,=Input_Conversão!U3,None,Sim,=Input_Conversão!J3,13/05/2024,24/05/2024,=ESQUERDA(Input_Conversão!P3; 2),= Input_Conversão!T3 * 24,=Input_Conversão!AF3,=Input_Conversão!AG3
2,=EXT.TEXTO(Input_Conversão!B4; 42; 2),=Input_Conversão!A4,=Input_Conversão!AE4,None,=EXT.TEXTO(Input_Conversão!B4; 46; 5000),=Input_Conversão!C4,=Input_Conversão!AD4,=Input_Conversão!AC4,=Input_Conversão!G4,=Input_Conversão!U4,None,Sim,=Input_Conversão!J4,13/05/2024,24/05/2024,=ESQUERDA(Input_Conversão!P4; 2),= Input_Conversão!T4 * 24,=Input_Conversão!AF4,=Input_Conversão!AG4
3,=EXT.TEXTO(Input_Conversão!B5; 42; 2),=Input_Conversão!A5,=Input_Conversão!AE5,None,=EXT.TEXTO(Input_Conversão!B5; 46; 5000),=Input_Conversão!C5,=Input_Conversão!AD5,=Input_Conversão!AC5,=Input_Conversão!G5,=Input_Conversão!U5,None,Sim,=Input_Conversão!J5,13/05/2024,24/05/2024,=ESQUERDA(Input_Conversão!P5; 2),= Input_Conversão!T5 * 24,=Input_Conversão!AF5,=Input_Conversão!AG5
4,=EXT.TEXTO(Input_Conversão!B6; 42; 2),=Input_Conversão!A6,=Input_Conversão!AE6,None,=EXT.TEXTO(Input_Conversão!B6; 46; 5000),=Input_Conversão!C6,=Input_Conversão!AD6,=Input_Conversão!AC6,=Input_Conversão!G6,=Input_Conversão!U6,None,Sim,=Input_Conversão!J6,13/05/2024,24/05/2024,=ESQUERDA(Input_Conversão!P6; 2),= Input_Conversão!T6 * 24,=Input_Conversão!AF6,=Input_Conversão!AG6


## Update output worksheet

In [16]:
set_with_dataframe(output_wks, df_output, resize='True')

# **Update main Tasks table**

Get task worksheet

In [12]:
# Tasks worksheet
if 'tasks_wks' not in globals():
  tasks_wks = database.worksheet('Tarefas')

tasks_wks

<Worksheet 'Tarefas' id:1754267222>

Make relationship between Task and who is going to deliever that task based on old cells already filled.

In [19]:
output_values = output_wks.get_all_values()
df_output = pd.DataFrame(output_values, columns=output_values[0])

# This is a way to get only the non null rows. If done in a simpler way with tasks_wks.row_count for example can lead to
non_empty_tasks_rows = sum(1 for row in all_tasks_values if any(cell.strip() for cell in row))

df_output['Função'] = [f"=SEERRO(PROCV(I{row_number};Val!$E:$G;2;FALSO());"")" for row_number in range(non_empty_tasks_rows, non_empty_tasks_rows + len(df_output))]
update_df_with_relations(df_output, column_name_task='Task', column_to_update='Entrega', worksheet=tasks_wks)
update_df_with_relations(df_output, column_name_task='Task', column_to_update='Produto', worksheet=tasks_wks)


{'Suporte técnico': ['Supervisão Tecnica'], 'Gestão do projeto': ['Gestão do Projeto'], 'Acompanhamento do time e entendimento do negócio': ['Acompanhamento do time Edenred'], 'Reuniões de sprint': ['Reunião de Projetos'], 'Acompanhamento do time Edenred': ['Acompanhamento do time Edenred'], 'Gestão de projeto': ['Gestão do Projeto'], 'Acompanhamento do time Edenred e passagem de conhecimento Thalis': ['Acompanhamento do time Edenred'], 'Acompanhamento do time Edenred e entendimento de negócio': ['Acompanhamento do time Edenred'], 'Reuniões de projeto': ['Reunião de Projetos', 'Reuniões'], 'Acompanhamento de projeto': ['Acompanhamento do time Edenred'], 'Supervisão técnica': ['Supervisão Tecnica'], 'Reuniões do projeto': ['Reunião de Projetos'], 'Relatório SLA': ['Melhorias e atualização nos relatorios'], 'Onboarding do projeto': ['Melhorias e atualização nos relatorios'], 'Acompanhamento time Edenred': ['Acompanhamento do time Edenred'], 'Onboarding sustentação - Thalis': ['Melhorias 

In [20]:
# output_values = output_wks.get_all_values()
# df_output = pd.DataFrame(output_values, columns=output_values[0])
# all_tasks_values = tasks_wks.get_values()

# # This is a way to get only the non null rows. If done in a simpler way with tasks_wks.row_count for example can lead to
# non_empty_tasks_rows = sum(1 for row in all_tasks_values if any(cell.strip() for cell in row))

# csc_valid_sheets = [
#     '[Edenred CSC] Controle de projeto ATIVO',
#     '[Edenred CSC] Controle de projeto ATIVO - v2',
#     'Cópia de [Edenred CSC] Controle de projeto ATIVO v2'
# ]

# ticket_log_valid_sheets = [
#     '[Edenred Ticket log] Controle de projeto ATIVO - v2',
#     'Cópia de [Edenred Ticket log] Controle de projeto ATIVO - v2'
# ]

# gta_valid_sheets = [
#     '[Edenred GTA] Controle de projeto ATIVO',
#     '[Edenred GTA - Sustentação PMO] Controle de projeto ATIVO',
#     '[Edenred GTA - Motor de Crédito] Controle de projeto ATIVO'
# ]

# df_output['Função'] = [f"=SEERRO(PROCV(I{row_number};Val!$E:$G;2;FALSO());"")" for row_number in range(non_empty_tasks_rows, non_empty_tasks_rows + len(df_output))]

# if spreedsheet_title in csc_valid_sheets:
#     update_df_with_relations(df_output, column_name_task='Task', column_to_update='Entrega', worksheet=tasks_wks)
#     update_df_with_relations(df_output, column_name_task='Task', column_to_update='Produto', worksheet=tasks_wks)
#     # update_df_with_relations(df_output, column_name_task='Task', column_to_update='Sub Entregas', worksheet=tasks_wks)
#     # update_df_with_relations(df_output, column_name_task='Task', column_to_update='Frente de trabalho', worksheet=tasks_wks)

# elif spreedsheet_title in ticket_log_valid_sheets:
#     update_df_with_relations(df_output, column_name_task='Task', column_to_update='Entrega', worksheet=tasks_wks)
#     update_df_with_relations(df_output, column_name_task='Task', column_to_update='Produto', worksheet=tasks_wks)
#     # update_df_with_relations(df_output, column_name_task='Task', column_to_update='Sub Entregas', worksheet=tasks_wks)

# elif spreedsheet_title in gta_valid_sheets:
#     update_df_with_relations(df_output, column_name_task='Task', column_to_update='Entrega', worksheet=tasks_wks)
#     update_df_with_relations(df_output, column_name_task='Task', column_to_update='Produto', worksheet=tasks_wks)

#     # =SE(OU(L427="Vinicius Pereira Fernandes";L427="Níkolas Santos Alves");"SRE";"Analytics")

In [21]:
df_output.head()

,Sprint,ID,CTI,Classificação,Task,Descricao,Entrega,Produto,Responsavel,Marcador,Função,Bitrix?,Status,Inicio,Conclusao,Tempo estimado (horas),Tempo Efetivo (horas),Tempo Efetivo Repasse (horas),Id task atrasada
0,Sprint,ID,CTI,Classificação,Task,Descricao,Entrega,Produto,Responsavel,Marcador,=SEERRO(PROCV(I1049;Val!$E:$G;2;FALSO());),Bitrix?,Status,Inicio,Conclusao,Tempo estimado (horas),Tempo Efetivo (horas),Tempo Efetivo Repasse (horas),Id task atrasada
1,51,294772,06.04.01-99,,Reuniões de projeto,,Reuniões,Gestão,Thalis Pasolini Trabach,,=SEERRO(PROCV(I1050;Val!$E:$G;2;FALSO());),Sim,Pending,13/05/2024,24/05/2024,08,"0,00",,
2,51,294774,06.04.01-99,,Reuniões de projeto,,Reuniões,Gestão,Juliana Gabriela dos Santos,,=SEERRO(PROCV(I1051;Val!$E:$G;2;FALSO());),Sim,In Progress,13/05/2024,24/05/2024,08,"7,96",,
3,51,294778,06.04.01-99,,Reuniões de projeto,,Reuniões,Gestão,Thaís Félix dos Santos,,=SEERRO(PROCV(I1052;Val!$E:$G;2;FALSO());),Sim,In Progress,13/05/2024,24/05/2024,08,"3,58",,
4,51,294780,06.04.01-99,,Reuniões de projeto,,Reuniões,Gestão,Hevans Vinicius Pereira,,=SEERRO(PROCV(I1053;Val!$E:$G;2;FALSO());),Sim,In Progress,13/05/2024,24/05/2024,08,"5,78",,


Functions: The idea is to use the procv formula to obtain the functions of each person responsible. We need to get the length of the task sheet and increase each row until we get the size of the output dataframe. This way we will have a perfect formula.

Update: to be able to update to the last rows of the table, we need to use apprend_rows. This method needs values to be list os lists. Also, we need [value_input_option](https://developers.google.com/sheets/api/reference/rest/v4/ValueInputOption?hl=pt-br) to be USER_ENTERED. If not, the formulas and numbers are will be inputed as strings.

In [22]:
# Verifique se existem valores para adicionar
if len(output_values) > 1:  # Verifica se há pelo menos uma linha de dados (ignora o cabeçalho)
    # Convert DataFrame to list of lists
    values_to_append = df_output.values.tolist()
    tasks_wks.append_rows(values=values_to_append[1:], table_range='A1', value_input_option='USER_ENTERED', insert_data_option='INSERT_ROWS')

    print(f'Valores adicionados em {tasks_wks.title}.')
else:
    print('Nenhum valor para adicionar à tabela de Tarefas.')


Valores adicionados em Tarefas.


## Deal with blank values

In [23]:
# data = tasks_wks.get_all_values()

# # Converter os dados em um dataframe do pandas
# df_tasks = pd.DataFrame(data[1:], columns=data[0])
# # df_entregas = df_tasks[['Entrega']].tail(len(df_output) - 1)
# # df_entregas
# df_tasks

In [24]:
def update_null_cells_with_formula(worksheet, df_tasks, column_name, formula_template):
    # Verificar se há valores vazios ou NaN na coluna especificada
    df_update_column = df_tasks[[column_name]].tail(len(df_output) - 1)

    null_values = df_update_column[column_name].apply(lambda x: x == "" or pd.isna(x))
    print("Null values found: ")
    print(null_values)
    cell_updates = []

    # Inserir a fórmula nas células vazias
    for index, row in df_update_column[null_values].iterrows():
        linha_excel = index + 2  # Compensando a linha do cabeçalho e base 0
        formula = formula_template.format(linha_excel)
        df_update_column.at[index, column_name] = formula

        # Adicionar a atualização à lista
        cell_updates.append((linha_excel, formula))
    print(cell_updates)
    # Atualizar o worksheet com as novas fórmulas
    if cell_updates:

        # Obter a faixa de células
        start_row = min(cell_updates, key=lambda x: x[0])[0]
        end_row = max(cell_updates, key=lambda x: x[0])[0]
        col_index = df_tasks.columns.get_loc(column_name) + 1

        cell_range = worksheet.range(start_row, col_index, end_row, col_index)
        print(column_name)
        print(col_index)
        # Atualizar os valores das células no intervalo
        for cell in cell_range:
            for update in cell_updates:
                if cell.row == update[0]:
                    cell.value = update[1]

        print(cell_range)

        # Atualizar as células no worksheet
        worksheet.update_cells(cell_range, value_input_option='USER_ENTERED')

    print(f'Fórmulas atualizadas na coluna {column_name}.')

    # Atualizar o dataframe original com as novas fórmulas
    df_tasks.update(df_tasks[[column_name]])

In [25]:
data = tasks_wks.get_all_values()

# Converter os dados em um dataframe do pandas
df_tasks = pd.DataFrame(data[1:], columns=data[0])

In [26]:
# Atualizar as células nulas na coluna 'Entrega'
update_null_cells_with_formula(
    worksheet=tasks_wks,
    df_tasks=df_tasks,
    column_name='Entrega',
    formula_template='=SEERRO(PROCV(E{}; Produtos_Entregas!$B$2:$B$8; 1); "")'
)

# Atualizar as células nulas na coluna 'Produto'
update_null_cells_with_formula(
    worksheet=tasks_wks,
    df_tasks=df_tasks,
    column_name='Produto',
    formula_template='=INDIRETO("Produtos_Entregas!A"&CORRESP(G{};Produtos_Entregas!B:B;0))'
)


Null values found: 
1048    False
1049    False
1050    False
1051    False
1052    False
1053    False
1054    False
1055    False
1056    False
1057    False
1058    False
1059    False
1060    False
1061    False
1062    False
1063    False
1064    False
1065    False
1066    False
1067    False
1068    False
1069    False
1070    False
1071    False
Name: Entrega, dtype: bool
[]
Fórmulas atualizadas na coluna Entrega.
Null values found: 
1048    False
1049    False
1050    False
1051    False
1052    False
1053    False
1054    False
1055    False
1056    False
1057    False
1058    False
1059    False
1060    False
1061    False
1062    False
1063    False
1064    False
1065    False
1066    False
1067    False
1068    False
1069    False
1070    False
1071    False
Name: Produto, dtype: bool
[]
Fórmulas atualizadas na coluna Produto.


# **Update Hours table**
